In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import pandas as pd
import os

In [ ]:
#loading osprey data
osprey_path = '/content/drive/MyDrive/ml_climate_project_data/osprey/'
csv_files = [f for f in os.listdir(osprey_path) if f.endswith('.csv')]
osprey_df = pd.concat([pd.read_csv(os.path.join(osprey_path, file)) for file in sorted(csv_files)], ignore_index = True)

#getting rid of missing / non-numeric values
osprey_df['OBSERVATION COUNT'] = pd.to_numeric(osprey_df['OBSERVATION COUNT'], errors = 'coerce')
osprey_df = osprey_df.dropna(subset = ['OBSERVATION COUNT'])
osprey_df = osprey_df[['OBSERVATION DATE', 'OBSERVATION COUNT', 'COUNTY']]

#loading bald_eagle data
eagle_path = '/content/drive/MyDrive/ml_climate_project_data/bald_eagle/'
csv_files = [f for f in os.listdir(eagle_path) if f.endswith('.csv')]
bald_eagle_df = pd.concat([pd.read_csv(os.path.join(eagle_path, file)) for file in sorted(csv_files)], ignore_index = True)

#getting rid of missing / non-numeric values
bald_eagle_df['OBSERVATION COUNT'] = pd.to_numeric(bald_eagle_df['OBSERVATION COUNT'], errors = 'coerce')
bald_eagle_df = bald_eagle_df.dropna(subset = ['OBSERVATION COUNT'])
bald_eagle_df = bald_eagle_df[['OBSERVATION DATE', 'OBSERVATION COUNT', 'COUNTY']]

#fish data
fish_df = pd.read_csv('/content/drive/MyDrive/ml_climate_project_data/fishdata.csv')

#temperature data
temp_df = pd.read_csv('/content/drive/MyDrive/ml_climate_project_data/tempdata.csv')
temp_df = temp_df[temp_df['Anomaly'] != -999]
temp_df['year'] = temp_df['Date'].astype(str).str[:4].astype(int)
temp_df['month'] = temp_df['Date'].astype(str).str[4:].astype(int)

#combine and join data
osprey_df["bird_type"] = "Osprey"
bald_eagle_df["bird_type"] = "Bald Eagle"
bird_df = pd.concat([osprey_df, bald_eagle_df], ignore_index = True)

bird_df['OBSERVATION DATE'] = pd.to_datetime(bird_df['OBSERVATION DATE'])
bird_df['year'] = bird_df['OBSERVATION DATE'].dt.year
bird_df['month'] = bird_df['OBSERVATION DATE'].dt.month

#combine with temperature data
bird_temp_df = bird_df.merge(temp_df[['year', 'month', 'Anomaly']], on = ['year', 'month'], how = 'left')

#combine with fish data
combined_df = bird_temp_df.merge(fish_df, on = 'year', how = 'left')
final_df = combined_df.dropna()
final_df = final_df.rename(columns = {"Anomaly": "temperature", "OBSERVATION COUNT": "bird_count", "Fish_Population": "fish_population"})

#merge data
final_df['OBSERVATION DATE'] = pd.to_datetime(final_df['OBSERVATION DATE'])
monthly_summary = final_df.groupby(['year', 'month', 'bird_type']).agg({'bird_count': 'sum', 'temperature': 'mean', 'runsize': 'mean', 'spawners': 'mean'}).reset_index()
bird_pivot = monthly_summary.pivot_table(index = ['year', 'month'], columns = 'bird_type', values = 'bird_count', fill_value = 0).reset_index()
bird_pivot.columns.name = None
bird_pivot = bird_pivot.rename(columns = {'Osprey': 'osprey_count', 'Bald Eagle': 'bald_eagle_count'})
temp_fish = monthly_summary.groupby(['year', 'month']).agg({'temperature': 'mean', 'runsize': 'mean', 'spawners': 'mean'}).reset_index()
merged_df = pd.merge(bird_pivot, temp_fish, on = ['year', 'month'], how = 'left')

In [11]:
#lagging data
merged_df = merged_df.sort_values(by = ['year', 'month']).reset_index(drop = True)

#lag by one month
merged_df['osprey_count_lag1'] = merged_df['osprey_count'].shift(1)
merged_df['temp_lag1'] = merged_df['temperature'].shift(1)
merged_df['fish_lag1'] = merged_df['runsize'].shift(1)

#lag by two months
merged_df['osprey_count_lag2'] = merged_df['osprey_count'].shift(2)
merged_df['temp_lag2'] = merged_df['temperature'].shift(2)
merged_df['fish_lag2'] = merged_df['runsize'].shift(2)

#lag by 3 months
merged_df['osprey_count_lag3'] = merged_df['osprey_count'].shift(3)
merged_df['temp_lag3'] = merged_df['temperature'].shift(3)
merged_df['fish_lag3'] = merged_df['runsize'].shift(3)

#rolling average by 3 months
merged_df['osprey_count_roll3'] = merged_df['osprey_count'].rolling(window = 3).mean()
merged_df['temp_roll3'] = merged_df['temperature'].rolling(window = 3).mean()
merged_df['fish_roll3'] = merged_df['runsize'].rolling(window = 3).mean()

merged_df = merged_df.dropna()